In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2443, done.
remote: Total 2443 (delta 0), reused 0 (delta 0), pack-reused 2443
Receiving objects: 100% (2443/2443), 8.13 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (1570/1570), done.


In [2]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 20.6 MB/s 
     |████████████████████████████████| 1.7 MB 43.6 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 180 kB 67.9 MB/s 
     |████████████████████████████████| 140 kB 67.1 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=d2c18eb4ed5a3df164f28a898288cd8ed7e618498a8d560c43c145dbf69b5e4e
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=061d2a7d27ccbf019036a048dc9d0f37c0b7de20e1c7f2df5c4bf9aaab527536
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl

In [13]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 14.7 MB/s 
     |████████████████████████████████| 948 kB 54.2 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=72e99267eee32753411612b6a9035edc024b398a9d4688ef5c2ea058e2c5eef9
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


# Dataset


In [4]:
!bash ./datasets/download_cyclegan_dataset.sh vangogh2photo

Streaming output truncated to the last 5000 lines.
  inflating: ./datasets/vangogh2photo/trainB/2016-08-26 19:25:29.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2016-02-20 16:54:25.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-05-19 19:43:44.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-10-26 21:18:58.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2014-07-19 15:39:53.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-06-13 21:21:54.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-06-04 14:10:33.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2014-06-18 12:46:34.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2016-03-17 18:07:22.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-01-24 17:24:29.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2016-01-27 12:45:41.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2016-06-14 05:57:28.jpg  
  inflating: ./datasets/vangogh2photo/trainB/2015-12-24 01:48:36.jpg  
  inflating: ./datasets/va

In [6]:
os.chdir("/content/")

In [11]:
os.getcwd()

'/content'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np

class StyleTransDataset(Dataset):
    def __init__(self, root_photo, root_vango, transform=None):
        self.root_photo = root_photo
        self.root_vango = root_vango
        self.transform = transform

        self.photo_images = os.listdir(root_photo)
        self.vango_images = os.listdir(root_vango)
        self.length_dataset = max(len(self.photo_images), len(self.vango_images)) # 1000, 1500
        self.photo_len = len(self.photo_images)
        self.vango_len = len(self.vango_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        photo_img = self.photo_images[index % self.photo_len]
        vango_img = self.vango_images[index % self.vango_len]

        photo_path = os.path.join(self.root_photo, photo_img)
        vango_path = os.path.join(self.root_vango, vango_img)

        photo_img = np.array(Image.open(photo_path).convert("RGB"))
        vango_img = np.array(Image.open(vango_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=photo_img, image0=vango_img)
            photo_img = augmentations["image"]
            vango_img = augmentations["image0"]

        return photo_img, vango_img

# Model

In [2]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature==features[-1] else 2))
            in_channels = feature
        layers.append(nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, img_channels, num_features = 64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
                ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
                ConvBlock(num_features*2, num_features*1, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )

        self.last = nn.Conv2d(num_features*1, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "vangogh2photo"
VAL_DIR = "data/val"
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
LAMBDA_IDENTITY = 0.1
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 10
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genh.pth.tar"
CHECKPOINT_GEN_Z = "genz.pth.tar"
CHECKPOINT_CRITIC_H = "critich.pth.tar"
CHECKPOINT_CRITIC_Z = "criticz.pth.tar"
LOAD_GEN_H = "/content/drive/MyDrive/cyclegan/12170750/genh.pth.tar"
LOAD_GEN_Z = "/content/drive/MyDrive/cyclegan/12170750/genz.pth.tar"
LOAD_CRITIC_H = "/content/drive/MyDrive/cyclegan/12170750/critich.pth.tar"
LOAD_CRITIC_Z = "/content/drive/MyDrive/cyclegan/12170750/criticz.pth.tar"


transforms = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
     ],
    additional_targets={"image0": "image"},
)

In [7]:
import random
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, "drive/MyDrive/cache17/"+filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [8]:
import sys
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image


def train_fn(disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler, epoch):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (photo, vango) in enumerate(loop):
        photo = photo.to(DEVICE)
        vango = vango.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():
            fake_vango = gen_H(photo)
            D_H_real = disc_H(vango)
            D_H_fake = disc_H(fake_vango.detach())
            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()
            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_photo = gen_Z(vango)
            D_Z_real = disc_Z(photo)
            D_Z_fake = disc_Z(fake_photo.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss)/2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_vango)
            D_Z_fake = disc_Z(fake_photo)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_photo = gen_Z(fake_vango)
            cycle_vango = gen_H(fake_photo)
            cycle_photo_loss = l1(photo, cycle_photo)
            cycle_vango_loss = l1(vango, cycle_vango)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_photo = gen_Z(photo)
            identity_vango = gen_H(vango)
            identity_photo_loss = l1(photo, identity_photo)
            identity_vango_loss = l1(vango, identity_vango)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_photo_loss * LAMBDA_CYCLE
                + cycle_vango_loss * LAMBDA_CYCLE
                + identity_vango_loss * LAMBDA_IDENTITY
                + identity_photo_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 200 == 0:
            save_image(fake_vango*0.5+0.5, f"drive/MyDrive/cache17/vango_{idx}_{epoch}.png")
            save_image(fake_photo*0.5+0.5, f"drive/MyDrive/cache17/photo_{idx}_{epoch}.png")

        loop.set_postfix(H_real=H_reals/(idx+1), H_fake=H_fakes/(idx+1))





In [9]:

disc_H = Discriminator(in_channels=3).to(DEVICE)
disc_Z = Discriminator(in_channels=3).to(DEVICE)
gen_Z = Generator(img_channels=3, num_residuals=9).to(DEVICE)
gen_H = Generator(img_channels=3, num_residuals=9).to(DEVICE)
opt_disc = optim.Adam(
    list(disc_H.parameters()) + list(disc_Z.parameters()),
    lr=LEARNING_RATE,
    betas=(0.5, 0.999),
)

opt_gen = optim.Adam(
    list(gen_Z.parameters()) + list(gen_H.parameters()),
    lr=LEARNING_RATE,
    betas=(0.5, 0.999),
)

L1 = nn.L1Loss()
mse = nn.MSELoss()

if LOAD_MODEL:
    load_checkpoint(
        LOAD_GEN_H, gen_H, opt_gen, LEARNING_RATE,
    )
    load_checkpoint(
        LOAD_GEN_Z, gen_Z, opt_gen, LEARNING_RATE,
    )
    load_checkpoint(
        LOAD_CRITIC_H, disc_H, opt_disc, LEARNING_RATE,
    )
    load_checkpoint(
        LOAD_CRITIC_Z, disc_Z, opt_disc, LEARNING_RATE,
    )

dataset = StyleTransDataset(
    root_vango=TRAIN_DIR+"/trainA", root_photo=TRAIN_DIR+"/trainB", transform=transforms
)

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):
    train_fn(disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler, epoch)

    if SAVE_MODEL:
        save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
        save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
        save_checkpoint(disc_H, opt_disc, filename=CHECKPOINT_CRITIC_H)
        save_checkpoint(disc_Z, opt_disc, filename=CHECKPOINT_CRITIC_Z)


=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint


100%|██████████| 6287/6287 [1:04:16<00:00,  1.63it/s, H_fake=0.319, H_real=0.671]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 6287/6287 [1:04:15<00:00,  1.63it/s, H_fake=0.318, H_real=0.672]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


...

=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 6287/6287 [1:04:14<00:00,  1.63it/s, H_fake=0.301, H_real=0.689]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


  6%|▌         | 379/6287 [03:53<1:00:35,  1.63it/s, H_fake=0.288, H_real=0.7]


KeyboardInterrupt: ignored

In [34]:
LOAD_GEN_H = "/content/drive/MyDrive/cyclegan/12161142/genh.pth.tar"
LOAD_GEN_Z = "/content/drive/MyDrive/cyclegan/12161142/genz.pth.tar"

load_checkpoint(
    LOAD_GEN_H, gen_H, opt_gen, LEARNING_RATE,
)
load_checkpoint(
    LOAD_GEN_Z, gen_Z, opt_gen, LEARNING_RATE,
)


=> Loading checkpoint
=> Loading checkpoint


In [27]:
def test_fn(disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (photo, vango) in enumerate(loop):
        photo = photo.to(DEVICE)
        vango = vango.to(DEVICE)

        with torch.cuda.amp.autocast():
            fake_vango = gen_H(photo)
            fake_photo = gen_Z(vango)
            #cycle_photo = gen_Z(photo)
            #cycle_vango = gen_H(vango)
            #recon_vango = gen_H(fake_photo)
            #recon_photo = gen_Z(fake_vango)
        
        save_image(fake_vango*0.5+0.5, f"drive/MyDrive/cache18/vangofake_{idx}.png")
        save_image(fake_photo*0.5+0.5, f"drive/MyDrive/cache18/photofake_{idx}.png")
        #save_image(cycle_vango*0.5+0.5, f"drive/MyDrive/cache18/vangocyc.png")
        #save_image(cycle_photo*0.5+0.5, f"drive/MyDrive/cache18/photocyc.png")
        #save_image(recon_vango*0.5+0.5, f"drive/MyDrive/cache18/vangore.png")
        #save_image(recon_photo*0.5+0.5, f"drive/MyDrive/cache18/photore.png")

        loop.set_postfix(H_real=H_reals/(idx+1), H_fake=H_fakes/(idx+1))

In [35]:
val_dataset = StyleTransDataset(
    root_vango="testA", root_photo="testB", transform=transforms
)
val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
)
test_fn(disc_H, disc_Z, gen_Z, gen_H, val_loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler)

100%|██████████| 8/8 [00:00<00:00, 10.40it/s, H_fake=0, H_real=0]
